In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Import libraries
import pandas as pd
import os
from pathlib import Path
from tqdm import tqdm
import yaml
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np
from PIL import Image, ExifTags
import torch

In [3]:
# INPUT_DIRS
INPUT_DATA_DIR = Path('dataset')
## Drop the Folder if it already exists
DATASETS_DIR = Path('dataset')
# Image & labels directory
TRAIN_IMAGES_DIR = DATASETS_DIR / 'images' / 'train'
TRAIN_LABELS_DIR = DATASETS_DIR / 'labels'/ 'train'
TEST_IMAGES_DIR = DATASETS_DIR / 'images' / 'test'
VAL_IMAGES_DIR = DATASETS_DIR / 'images' /'val'
VAL_LABELS_DIR = DATASETS_DIR / 'labels' /'val'

# Load train and test files
train = pd.read_csv(INPUT_DATA_DIR / 'Train_df.csv')
val = pd.read_csv(INPUT_DATA_DIR / 'Val_df.csv')
test = pd.read_csv(INPUT_DATA_DIR / 'Test.csv')
ss = pd.read_csv(INPUT_DATA_DIR / 'SampleSubmission.csv')

class_map = {cls: i for i, cls in enumerate(sorted(train['class'].unique().tolist()))}
# Strip any spacing from the class item and make sure that it is a str
train['class'] = train['class'].str.strip()

# Map {'healthy': 2, 'cssvd': 1, anthracnose: 0}
train['class_id'] = train['class'].map(class_map)

train_df = train
val_df = val

# Create a data.yaml file required by yolo
class_names = sorted(train['class'].unique().tolist())
num_classes = len(class_names)
data_yaml = {
	"path" : str(DATASETS_DIR.absolute()),
	'train': str(TRAIN_IMAGES_DIR.absolute()),
	'val': str(VAL_IMAGES_DIR.absolute()),
	'test': str(TEST_IMAGES_DIR.absolute()),
	'nc': num_classes,
	'names': class_names
}

val_image_names = [str(Path(name).stem) for name in val_df['Image_ID'].unique()]
train_image_names = [str(Path(name).stem) for name in train['ImagePath'].unique()]

In [4]:
val_df['Image_ID'].nunique()

830

In [5]:
from glob import glob

# Validate the model on the validation set
BEST_PATH = sorted(glob("zindi_challenge_cacao_stage2/train*/weights/best.pt"))[-1]
# BEST_PATH = "zindi_challenge_cacao/train3/weights/best.pt"
BEST_PATH

'zindi_challenge_cacao_stage2/train42/weights/best.pt'

In [6]:
# Validate the model on the validation set
BEST_CFG = sorted(glob("zindi_challenge_cacao_stage2/train*/args.yaml"))[-1]
# BEST_CFG = "zindi_challenge_cacao/train3/args.yaml"
BEST_CFG

'zindi_challenge_cacao_stage2/train42/args.yaml'

In [7]:
# Load the trained YOLO model
model = YOLO(BEST_PATH).eval()

In [8]:
for flag, v in ExifTags.TAGS.items():
	if v == "Orientation":
		break


def load_image_(filepath):
	image = Image.open(filepath)
	# return image

	exif = image._getexif()
	if exif is None:
		return image

	orientation_value = exif.get(flag, None)

	if orientation_value == 3:
		image = image.rotate(180, expand=True)
	elif orientation_value == 6:
		image = image.rotate(270, expand=True)
	elif orientation_value == 8:
		image = image.rotate(90, expand=True)
	return image

from ultralytics.utils.patches import imread
import cv2

def load_image(filepath):
	# return load_image_(filepath)
	return imread(filepath, cv2.IMREAD_COLOR)


flag

274

In [9]:
import yaml
with open(BEST_CFG, 'r') as f:
	cfg: dict = yaml.safe_load(f)
	print(cfg)

{'task': 'detect', 'mode': 'train', 'model': 'yolo11l.pt', 'data': 'data.yaml', 'epochs': 30, 'time': 2.5, 'patience': 20, 'batch': 8, 'imgsz': 1024, 'save': True, 'save_period': -1, 'cache': False, 'device': 'cuda:0', 'workers': 4, 'project': 'zindi_challenge_cacao_stage2', 'name': 'train42', 'exist_ok': False, 'pretrained': True, 'optimizer': 'auto', 'verbose': True, 'seed': 0, 'deterministic': True, 'single_cls': False, 'rect': False, 'cos_lr': True, 'close_mosaic': 10, 'resume': False, 'amp': True, 'fraction': 1.0, 'profile': False, 'freeze': None, 'multi_scale': False, 'overlap_mask': True, 'mask_ratio': 4, 'dropout': 0.21625404864404263, 'val': True, 'split': 'val', 'save_json': False, 'conf': None, 'iou': 0.6656773157840741, 'max_det': 30, 'half': True, 'dnn': False, 'plots': True, 'source': None, 'vid_stride': 1, 'stream_buffer': False, 'visualize': False, 'augment': True, 'agnostic_nms': True, 'classes': None, 'retina_masks': False, 'embed': None, 'show': False, 'save_frames':

In [10]:
# Batch size for predictions
batch_size = 16

cfg["device"] = "cuda:1"
cfg["batch"] = batch_size
cfg["conf"] = 0.0
cfg["verbose"] = False

cfg.pop("source", None)
# cfg.pop("batch_size")
cfg.pop("visualize", None)
cfg.pop("data", None)
cfg.pop("name", None)
# cfg.pop("half", None)

cfg["model"] = "predict"

keys = list(cfg.keys())
for col in keys:
    if (
        "show" in col
        or "save" in col
        or "freeze" in col
        # or "nms" in col
        # or "multi_scale" in col
        or "plot" in col
        or "aug" in col
        or "drop" in col
    ):
        cfg.pop(col)

In [11]:
# Path to the test images directory
test_dir_path = VAL_IMAGES_DIR

# Get a list of all image files in the test directory
image_files = [i for i in os.listdir(test_dir_path) if not i.endswith(".npy")]

# Initialize an empty list to store the results for all images
all_data = []

# Initialize an empty list to store the results for all images
all_data = []


with torch.no_grad():
	# Process images in batches
	for i in tqdm(range(0, len(image_files), batch_size)):
		batch_files = image_files[i:i + batch_size]
		batch_images = [load_image(os.path.join(test_dir_path, img_file)) for img_file in batch_files]

		# Make predictions on the batch of images
		results = model.predict(
			batch_images,
			**cfg,
		)

		# Iterate through each result in the batch
		for img_file, result in zip(batch_files, results):
			boxes = result.boxes.xyxy.tolist() if result.boxes else []  # Bounding boxes in xyxy format
			classes = result.boxes.cls.tolist() if result.boxes else []  # Class indices
			confidences = result.boxes.conf.tolist() if result.boxes else []  # Confidence scores
			names = result.names  # Class names dictionary

			if boxes:  # If detections are found
				for box, cls, conf in zip(boxes, classes, confidences):
					x1, y1, x2, y2 = box
					detected_class = names[int(cls)]  # Get the class name from the names dictionary

					# Add the result to the all_data list
					all_data.append(
						{
							"Image_ID": str(img_file),
							"class": detected_class,
							"confidence": conf,
							"ymin": y1,
							"xmin": x1,
							"ymax": y2,
							"xmax": x2,
						}
					)
			else:  # If no objects are detected
				all_data.append(
					{
						"Image_ID": str(img_file),
						"class": "None",
						"confidence": None,
						"ymin": None,
						"xmin": None,
						"ymax": None,
						"xmax": None,
					}
				)


100%|██████████| 52/52 [01:01<00:00,  1.18s/it]


In [12]:
# Convert the list to a DataFrame for all images
sub = pd.DataFrame(all_data)

In [13]:
sub.head()

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
0,ID_MYSxE2.jpg,healthy,0.816406,62.15625,366.56250,3109.406250,1907.71875
1,ID_MYSxE2.jpg,healthy,0.216675,0.00000,1759.50000,498.843750,2448.00000
2,ID_MYSxE2.jpg,healthy,0.171021,1.59375,1244.71875,591.281250,1834.40625
3,ID_MYSxE2.jpg,healthy,0.066589,2836.87500,325.12500,3264.000000,1249.50000
4,ID_MYSxE2.jpg,healthy,0.022415,0.00000,1820.06250,363.175781,2448.00000


In [14]:
sub.describe()

,confidence,ymin,xmin,ymax,xmax
count,24900.000000,24900.000000,24900.000000,24900.000000,24900.000000
mean,0.043981,635.425510,608.635987,1420.084606,1318.417118
std,0.150809,835.540673,737.785070,1146.753892,967.081059
min,0.000019,0.000000,0.000000,8.804198,0.000000
25%,0.000536,3.691406,19.687498,465.593750,555.184509
50%,0.001796,274.375000,357.093719,1080.000000,967.453094
75%,0.006958,992.460938,879.750000,2094.187500,1933.312378
max,0.919434,3950.625000,3937.499756,4128.000000,4128.000000


In [15]:
sub['class'].value_counts()

class
healthy        9336
cssvd          8798
anthracnose    6766
Name: count, dtype: int64

In [16]:
def load_yolo_labels(label_folder):
	label_data = {}
	label_folder = Path(label_folder)
	paths = [i for i in label_folder.glob("*") if i.suffix != ".npy"]

	for label_file in paths:
		with open(label_file, "r") as file:
			annotations = []
			for line in file:
				parts = line.strip().split()
				if len(parts) == 5:
					class_id, x_center, y_center, width, height = map(float, parts)
					annotations.append({
						"class_id": int(class_id),
						"x_center": x_center,
						"y_center": y_center,
						"width": width,
						"height": height
					})
				else:
					print(f"Skipping line in {label_file}: {line.strip()}")
			label_data[label_file.stem] = annotations
	# Convert the label data to a pandas DataFrame
	label_df = []
	for image_id, annotations in label_data.items():
		for annotation in annotations:
			label_df.append({
				"Image_ID": image_id,
				"class_id": annotation["class_id"],
				"x_center": annotation["x_center"],
				"y_center": annotation["y_center"],
				"width": annotation["width"],
				"height": annotation["height"]
			})

	label_df = pd.DataFrame(label_df)
	return label_df

# Example usage
label_folder = VAL_LABELS_DIR
labels = load_yolo_labels(label_folder)
labels.sample(5)

def yolo_to_bbox(image_folder, labels_df: pd.DataFrame):
	image_folder = Path(image_folder)
	converted_bboxes = []

	paths = [i for i in image_folder.glob("*") if i.suffix != ".npy"]
	for image_file in paths:
		image_id = image_file.stem
		if image_id not in labels_df['Image_ID'].values:
			converted_bboxes.append({
				"Image_ID": image_id,
				"class_id": -1,  # Indicating no label
				"xmin": None,
				"ymin": None,
				"xmax": None,
				"ymax": None
			})

	for _, row in labels_df.iterrows():
		all_ids = [i for i in image_folder.glob(f"{row['Image_ID']}*") if i.suffix != ".npy"]
		image_path = image_folder / f"{row['Image_ID']}"
		if all_ids:
			image_path = all_ids[0]

		if image_path.exists():
			img = load_image_(image_path)
			img_width, img_height = img.size

			x_center = row['x_center'] * img_width
			y_center = row['y_center'] * img_height
			width = row['width'] * img_width
			height = row['height'] * img_height

			x_min = x_center - (width / 2)
			y_min = y_center - (height / 2)
			x_max = x_center + (width / 2)
			y_max = y_center + (height / 2)

			converted_bboxes.append({
				"Image_ID": row['Image_ID'],
				"class_id": row['class_id'],
				"xmin": x_min,
				"ymin": y_min,
				"xmax": x_max,
				"ymax": y_max
			})
		else:
			print(f"Image {image_path} not found.")

	return pd.DataFrame(converted_bboxes)

# Example usage
converted_labels = yolo_to_bbox(VAL_IMAGES_DIR, labels)
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax
927,ID_liJHy8,1,114.000384,592.000000,1114.000896,1498.000384
498,ID_NK7BSD,0,128.999304,276.998400,2177.998200,2910.999168
966,ID_E85J7J,0,0.999936,224.000000,705.999360,920.000512
396,ID_WFB5Qa,2,85.000032,49.000016,252.999968,331.000176
298,ID_iXCKHh,2,1.000064,166.999872,95.000256,338.000000


In [17]:
converted_labels.describe()

,class_id,xmin,ymin,xmax,ymax
count,1454.000000,1454.000000,1454.000000,1454.000000,1454.000000
mean,1.215956,423.980715,428.680175,1446.945665,1797.552956
std,0.783603,589.500230,565.801693,1006.550337,1182.598521
min,0.000000,-0.002016,-0.002064,33.999840,53.000064
25%,1.000000,10.000128,27.000272,629.250093,858.500130
50%,1.000000,197.000168,187.499640,1187.999280,1507.999492
75%,2.000000,580.001202,613.749054,2249.749404,2796.998814
max,2.000000,3612.000672,3591.000000,4128.000000,4128.002064


In [18]:
converted_labels["Image_ID"].nunique()

830

In [19]:
converted_labels['class_id'].value_counts()

class_id
2    637
1    494
0    323
Name: count, dtype: int64

In [20]:
class_map

{'anthracnose': 0, 'cssvd': 1, 'healthy': 2}

In [21]:
id_class_map = {v: k for k, v in class_map.items()}
converted_labels['class'] = converted_labels['class_id'].map(id_class_map)
converted_labels['class'].value_counts()

class
healthy        637
cssvd          494
anthracnose    323
Name: count, dtype: int64

In [22]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
271,ID_n8PmEh,0,382.999968,59.998416,2301.999840,4127.997936,anthracnose
1079,ID_PR1tKH,2,0.000000,527.000544,1750.998816,3994.000416,healthy
941,ID_hUfLpX,0,1567.998612,1372.999632,2488.999788,3313.997616,anthracnose
645,ID_VaOlPl,1,39.999825,53.999460,809.999595,985.999500,cssvd
797,ID_qN1Fz6,1,1729.000368,319.002144,2284.999920,1112.000736,cssvd


In [23]:
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
3694,ID_TejZZZ.jpg,healthy,0.001796,987.187500,355.429688,1080.000000,727.734375
12438,ID_Nt40cj.jpg,cssvd,0.000178,0.000000,522.000000,165.250000,1264.000000
22187,ID_ANnwzq.jpg,healthy,0.001604,1.675781,0.063477,77.238274,22.508787
7268,ID_LSqjdW.jpg,cssvd,0.007233,927.562500,1345.125000,1711.687500,2750.812500
22153,ID_k0McRl.jpeg,cssvd,0.006489,0.000000,591.250000,374.062500,960.000000


In [24]:
sub.loc[:, "Image_ID"] = sub["Image_ID"].apply(lambda x: str(Path(x).stem))

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
21259,ID_jzPiFN,cssvd,0.003075,2297.531250,157.992188,3024.000000,1950.539062
10061,ID_iLqTcn,anthracnose,0.001065,0.156250,0.000000,154.453125,121.562500
22087,ID_z6gC1L,anthracnose,0.003172,81.632812,1624.593750,1348.453125,3096.000000


In [25]:
def convert_df(df: pd.DataFrame):
	df = df.copy().dropna()
	return {
		img_id: {
			"boxes": torch.tensor(raw[["xmin", "ymin", "xmax", "ymax"]].values, dtype=torch.float32),
			"scores": (
				torch.tensor(raw["confidence"].values, dtype=torch.float32)
				if "confidence" in raw.columns
				else None
			),
			"labels": torch.tensor(raw["class_id"].values, dtype=torch.int32),
		}
		for (img_id, ), raw in df.groupby(["Image_ID"])
	}

def default_value():
	return {
		"boxes": torch.empty((0, 4), dtype=torch.float32),
		"scores": torch.empty((0,), dtype=torch.float32),
		"labels": torch.empty((0,), dtype=torch.int32),
	}

def get_preds_data(preds, thr: float = 0.5):
	if thr is not None:
		preds = preds[preds["confidence"] >= thr]
	preds = convert_df(preds)
	d = default_value()
	return {i: preds.get(i, d) for i in converted_labels["Image_ID"].unique()}

In [26]:
converted_labels.isna().sum()

Image_ID    0
class_id    0
xmin        0
ymin        0
xmax        0
ymax        0
class       0
dtype: int64

In [27]:
ground_truth = convert_df(converted_labels)
ground_truth = {k: ground_truth[k] for k in converted_labels["Image_ID"].unique()}

len(ground_truth)

830

In [28]:
import torch

def calculate_iou_tensor(box1, box2):
	"""
	box1: [4], box2: [4]
	Format: [xmin, ymin, xmax, ymax]
	"""
	xA = torch.max(box1[0], box2[0])
	yA = torch.max(box1[1], box2[1])
	xB = torch.min(box1[2], box2[2])
	yB = torch.min(box1[3], box2[3])

	inter_area = torch.clamp(xB - xA, min=0) * torch.clamp(yB - yA, min=0)
	box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
	box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
	union_area = box1_area + box2_area - inter_area
	return inter_area / union_area if union_area > 0 else torch.tensor(0.0)

def evaluate_detection(predictions, ground_truths, iou_threshold=0.5, conf_threshold=0.0):
	"""
	predictions: list of dicts (len = batch size), each dict with 'boxes', 'scores', 'labels'
	ground_truths: list of dicts with 'boxes', 'labels'
	"""
	TP = 0
	FP = 0
	FN = 0

	for preds, gts in zip(predictions, ground_truths):
		pred_boxes = preds['boxes']
		pred_labels = preds['labels']
		pred_scores = preds['scores'] if preds['scores'] is not None else torch.ones(len(pred_boxes))

		gt_boxes = gts['boxes']
		gt_labels = gts['labels']
		matched_gt = set()

		for i in range(len(pred_boxes)):
			if pred_scores[i] < conf_threshold:
				continue
			pred_box = pred_boxes[i]
			pred_label = pred_labels[i]
			match_found = False

			for j in range(len(gt_boxes)):
				if j in matched_gt:
					continue
				if pred_label != gt_labels[j]:
					continue
				iou = calculate_iou_tensor(pred_box, gt_boxes[j])
				if iou >= iou_threshold:
					TP += 1
					matched_gt.add(j)
					match_found = True
					break
			if not match_found:
				FP += 1

		FN += len(gt_boxes) - len(matched_gt)

	precision = TP / (TP + FP) if (TP + FP) else 0.0
	recall = TP / (TP + FN) if (TP + FN) else 0.0
	f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) else 0.0
	accuracy = TP / (TP + FP + FN) if (TP + FP + FN) else 0.0

	return {
		'TP': TP,
		'FP': FP,
		'FN': FN,
		'Precision': precision,
		'Recall': recall,
		'F1 Score': f1_score,
		'Accuracy': accuracy
	}

In [29]:
sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax
7737,ID_KPCvjE,healthy,0.000612,0.000000,5.906250,1193.062500,1579.921875
3422,ID_bouUUL,healthy,0.008255,2820.312256,44.921871,3999.999756,1240.234253
6846,ID_RYn1TJ,cssvd,0.022934,295.441406,34.265625,1036.734375,504.421875


In [30]:
sub["class_id"] = sub["class"].map(class_map)

sub.sample(3)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
3709,ID_TejZZZ,cssvd,0.000142,504.140625,1.845703,1080.000000,394.716797,1
19635,ID_bg62Nn,anthracnose,0.000364,159.375000,0.000000,2978.718750,1224.000000,0
7259,ID_PHLxKY,anthracnose,0.000365,0.000000,473.484344,447.152313,2945.249756,0


In [31]:
predictions = get_preds_data(sub, None)

len(predictions)

830

In [32]:
for i in np.linspace(0.0, 0.95, 15):
	scores = evaluate_detection(
		predictions.values(),
		ground_truth.values(),
		iou_threshold=0.5,
		conf_threshold=i
	)
	print("Evaluation metric at:", i, " score :", scores)

Evaluation metric at: 0.0  score : {'TP': 1341, 'FP': 23559, 'FN': 113, 'Precision': 0.053855421686746986, 'Recall': 0.9222833562585969, 'F1 Score': 0.10176823252637171, 'Accuracy': 0.05361212169671771}
Evaluation metric at: 0.06785714285714285  score : {'TP': 1179, 'FP': 996, 'FN': 275, 'Precision': 0.5420689655172414, 'Recall': 0.8108665749656121, 'F1 Score': 0.649765775695784, 'Accuracy': 0.48122448979591836}
Evaluation metric at: 0.1357142857142857  score : {'TP': 1114, 'FP': 626, 'FN': 340, 'Precision': 0.6402298850574712, 'Recall': 0.7661623108665749, 'F1 Score': 0.6975579211020664, 'Accuracy': 0.5355769230769231}
Evaluation metric at: 0.20357142857142857  score : {'TP': 1085, 'FP': 457, 'FN': 369, 'Precision': 0.7036316472114138, 'Recall': 0.7462173314993122, 'F1 Score': 0.7242990654205608, 'Accuracy': 0.5677655677655677}
Evaluation metric at: 0.2714285714285714  score : {'TP': 1046, 'FP': 339, 'FN': 408, 'Precision': 0.7552346570397112, 'Recall': 0.71939477303989, 'F1 Score': 0

In [33]:
converted_labels.sample(5)

,Image_ID,class_id,xmin,ymin,xmax,ymax,class
276,ID_Myyluu,2,65.999856,65.999856,413.000016,292.999824,healthy
909,ID_UIsNO1,1,155.001168,1053.001152,2804.000976,2178.001728,cssvd
1260,ID_BW28Rb,2,200.000160,184.999680,821.000160,789.999360,healthy
209,ID_mb8nj1,0,120.001392,67.001760,2577.001392,4032.002016,anthracnose
1371,ID_md0MsD,2,0.000000,230.999680,235.999872,1018.999680,healthy


In [34]:
import torch
from torchmetrics.detection import MeanAveragePrecision


def compute_map(preds, targets):
    # Initialize the metric
    metric = MeanAveragePrecision()

    # Update metric with predictions and targets
    metric.update(preds, targets)

    # Compute the results
    result = metric.compute()

    return result


thrs = np.linspace(0.000001, 0.95, 15)
for i in thrs:
    preds = list(get_preds_data(sub, i).values())

    targets = list(ground_truth.values())

    # Compute mAP
    results = compute_map(preds, targets)

    # Print results
    print("mAP Results:", i, " - ", results)

mAP Results: 1e-06  -  {'map': tensor(0.4917), 'map_50': tensor(0.7597), 'map_75': tensor(0.5583), 'map_small': tensor(0.), 'map_medium': tensor(0.2528), 'map_large': tensor(0.4957), 'mar_1': tensor(0.3914), 'mar_10': tensor(0.6084), 'mar_100': tensor(0.6339), 'mar_small': tensor(0.), 'mar_medium': tensor(0.4313), 'mar_large': tensor(0.6376), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.06785807142857142  -  {'map': tensor(0.4742), 'map_50': tensor(0.7272), 'map_75': tensor(0.5424), 'map_small': tensor(0.), 'map_medium': tensor(0.2445), 'map_large': tensor(0.4777), 'mar_1': tensor(0.3897), 'mar_10': tensor(0.5645), 'mar_100': tensor(0.5645), 'mar_small': tensor(0.), 'mar_medium': tensor(0.3594), 'mar_large': tensor(0.5683), 'map_per_class': tensor(-1.), 'mar_100_per_class': tensor(-1.), 'classes': tensor([0, 1, 2], dtype=torch.int32)}
mAP Results: 0.13571514285714284  -  {'map': tensor(0.4628), 'map_50':

In [35]:
sub.to_csv('dataset/evaluations/validation.csv', index=False)

In [36]:
import pandas as pd

sub = pd.read_csv('dataset/evaluations/validation.csv')
sub.sample(5)

,Image_ID,class,confidence,ymin,xmin,ymax,xmax,class_id
21134,ID_znDE9u,cssvd,0.001567,0.000000,496.124969,1102.499878,2210.906006,1
14720,ID_oaam4S,cssvd,0.000163,0.000000,3.955078,155.698242,322.470703,1
8235,ID_B3knOq,cssvd,0.001065,6.890625,2980.687256,1134.984253,4032.000000,1
10990,ID_HTXcz3,healthy,0.005867,9.546874,0.000000,154.781235,32.347652,2
9877,ID_c2XesD,healthy,0.047333,4.976562,70.281242,178.546860,371.312469,2
